In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [9]:
#Noisy variables are removed from features
application_df.drop(["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"], 1, inplace = True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [23]:
# Determine which values to replace if counts are less than ...?
app_value_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = list(app_value_counts[app_value_counts < 4].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Determine which values to replace if counts are less than ..?
classification = application_df.CLASSIFICATION.value_counts()
replace_class = list(classification[classification < 1500].index)
replace_class

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [25]:
# Generate our categorical variable lists
categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
categories

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [26]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[categories]))
encode_df
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(categories)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index = True, right_index= True)
application_df.drop(categories, 1, inplace = True)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop("IS_SUCCESSFUL", 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
#model 1  
#add more neurons to each hidden layer 
nn1 = tf.keras.models.Sequential()

number_input_features = len(application_df.columns) - 1
hidden_layer1 = 15
hidden_layer2 = 8

nn.add(tf.keras.layers.Dense(units = hidden_layer1, input_dim = number_input_features, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = hidden_layer2, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

nn.summary()

# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                675       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 812
Trainable params: 812
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5870 - accuracy: 0.7141
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5604 - accuracy: 0.7269
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7289
Epoch 4/50
804/804 [

In [41]:
#second model 
#add a third hidden layer 
nn2 = tf.keras.models.Sequential()

number_input_features = len(application_df.columns) - 1
hidden_layer1 = 15
hidden_layer2 = 8

nn.add(tf.keras.layers.Dense(units = hidden_layer1, input_dim = number_input_features, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = hidden_layer2, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 4, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

nn.summary()

# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                675       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
 dense_3 (Dense)             (None, 15)                30        
                                                                 
 dense_4 (Dense)             (None, 8)                 128       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
 dense_6 (Dense)             (None, 15)               

804/804 [==============================] - 4s 4ms/step - loss: 0.6913 - accuracy: 0.5314
Epoch 20/50
804/804 [==============================] - 4s 4ms/step - loss: 0.6913 - accuracy: 0.5314
Epoch 21/50
804/804 [==============================] - 5s 6ms/step - loss: 0.6913 - accuracy: 0.5314
Epoch 22/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5314
Epoch 23/50
804/804 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 24/50
804/804 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 25/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 26/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5314
Epoch 27/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 28/50
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5314
Ep

In [34]:
#third model 
#change the optimizer  
nn2 = tf.keras.models.Sequential()

number_input_features = len(application_df.columns) - 1
hidden_layer1 = 15
hidden_layer2 = 8

nn.add(tf.keras.layers.Dense(units = hidden_layer1, input_dim = number_input_features, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = hidden_layer2, activation = "tanh"))
nn.add(tf.keras.layers.Dense(units = 4, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

nn.summary()

# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adagrad", metrics = ["accuracy"])

# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                675       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
 dense_3 (Dense)             (None, 15)                30        
                                                                 
 dense_4 (Dense)             (None, 8)                 128       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
 dense_6 (Dense)             (None, 15)               

In [39]:
#fourth model 
#change the optimizer  
nn2 = tf.keras.models.Sequential()

number_input_features = len(application_df.columns) - 1
hidden_layer1 = 50
hidden_layer2 = 20

nn.add(tf.keras.layers.Dense(units = hidden_layer1, input_dim = number_input_features, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = hidden_layer2, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

nn.summary()

# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                675       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
 dense_3 (Dense)             (None, 15)                30        
                                                                 
 dense_4 (Dense)             (None, 8)                 128       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
 dense_6 (Dense)             (None, 15)               

804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 30/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 31/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 32/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 33/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 34/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 35/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 36/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 37/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Epoch 38/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5314
Ep